In [16]:
from bb_behavior import plot
from bb_behavior import db as bbdb
import bb_behavior
import pandas as pd
import os
import sys
sys.path.append('/home/mi/rrszynka/mnt/janek/Beesbook-janek/Python-modules/') #For bee_helpers, file_helpers and cache
from bee_cache import Cache, CacheType, CacheFormat; c = Cache()
import datetime
from bb_behavior import db as bbdb
from skimage.morphology import rectangle, closing
import datetime
import numpy as np
from tqdm import tqdm
import random
#%%

In [39]:
detection_confidence_requirement = 0.99
# detection_confidence_requirement = 0.2
conf_string = str(detection_confidence_requirement).replace('.',"")
interval_size = 30
fps = 3

loc = '/home/mi/rrszynka/mnt/janek/caches/Other/'

# Get one of the pre-generated lists  
gaps = pd.read_csv(loc+'120_random_gaps_unmasked_unfiltered_seed_666_conf_' + conf_string + '.csv', parse_dates=['entry', 'entry_closest_detection_timestamp', 'exit', 'exit_closest_detection_timestamp'])

In [34]:
%%capture
# Load presence files to be able to refer to them when generating vids 
presences = c.load_multiple_day_caches(detection_confidence_requirement=detection_confidence_requirement) # 0.99 / 0.2

In [35]:
presences_bin = []
for date, presence in presences:
    presence_bin = binarize_presence(presence)
    presences_bin.append((date, presence_bin))

In [ ]:
#TODO: from 120_random_gaps_unmasked_unfiltered_seed_666_conf_099, .iloc[14] doesn't work (something is nan)A

In [41]:
row = gaps.iloc[17]
# generate_presence_video(row)

In [42]:
row

Unnamed: 0                                                 7087
age                                                          15
age_group                                                     3
bee_id                                                      993
cam_id_entry                                                  2
cam_id_exit                                                   2
entry                                       2016-07-26 03:09:00
entry_closest_detection_timestamp    2016-07-26 03:09:00.474956
entry_frame                                17316603647305620768
entry_x                                                 22.5931
entry_y                                                 77.4607
exit                                        2016-07-26 02:54:30
exit_closest_detection_timestamp     2016-07-26 02:54:30.093554
exit_frame                                 13130495958401725792
exit_x                                                  6.95881
exit_y                                  

In [36]:
# GENERATE MUTLIPLE VIDEOS
for i, row in gaps[:100].iterrows():
    try:
        generate_presence_video(row)
    except (ValueError, TypeError):
        print('error, skip one')
        continue

AttributeError: 'Series' object has no attribute 'hiveexit_distance_for_exit'

In [21]:
gaps.shape

(120, 23)

In [ ]:
# Would be nice to see the presence(s) in context, I guess?

In [ ]:
# plot_presence(presx)
# plot_presence(presbin)

In [31]:
#%% FUNCTIONS
def generate_presence_video(row):
    bee, timestamp, frame = row.bee_id, row.exit, row.exit_frame 
    cam_id = row.cam_id_exit
    dist = row.hiveexit_distance_for_exit 
    # ASK: can this be used given hiccups? or just take days w/o hiccups?
    interval_margin = 2
    frames_per_interval = interval_size*fps
    frame_margin = interval_margin*frames_per_interval #30s * 3FPS
    name = str(bee)+'_random_exit'
    path = '/home/mi/rrszynka/mnt/janek/caches/Videos/' + name + ".mp4"
    if os.path.isfile(path):
        print(str(bee) + ' - ' + str(timestamp) + ' already exists, returning with no result.')
        return
    video = plot.plot_bees(bee_ids=[bee],
                       frame_id=frame, frame_margin=frame_margin, # TEMP: should be frame_margin
                       path_alpha=None, bt_export=None, # '/home/mi/rrszynka/mnt/janek/caches/Videos/bt_export/'+name+".json",
                       plot_labels = True, plot_markers=True)

    intervals = len(video._frames)/(interval_margin*2)

    #TODO: check this conversion from timestamp to interval against the conversion that goes the other way
    middle_timestamp = timestamp

    secs = time_to_secs(middle_timestamp)
    interval_index_mid = int(secs/30)
    print(timestamp, interval_index_mid)
    start_interval = interval_index_mid - interval_margin

    _, presence_df = next(((d, p) for d , p in presences if d == timestamp.date()), None)
    _, bin_presence_df = next(((d, p) for d , p in presences_bin if d == timestamp.date()), None)


    pres_row = presence_df.loc[bee]
    bin_pres_row = bin_presence_df.loc[bee]

    for i, frame in enumerate(video._frames):
        interval_index = (i // frames_per_interval)
        #PRINT: pres_score, pres_bin, cam_id, bee_id (or save in file), exit_dist(would be nice)
        frame._title = "\n c: " + str(cam_id) +", d:" + str(int(dist)) + ",  s: " + str(pres_row[start_interval + interval_index]) + ",  p: " + str(bin_pres_row[start_interval + interval_index])
    video._crop_margin = None
    video.get_video(save_to_path='/home/mi/rrszynka/mnt/janek/caches/Videos/' + name + ".mp4")

In [27]:
def time_to_secs(time):
    return time.hour * 3600 + time.minute * 60 + time.second

    
def binarize_presence(presence_for_day_df):
    # TODO: running this on an already-binarized series will return all zeros,
    # (and that's not what we want) - changing the threshold solves it, but try sth else

    ys = presence_for_day_df.copy()
    index = ys.index
    if ys[ys>1].sum().sum() == 0:
        #Consider this already binarized, make no changes
        return ys

    ys[ys>90] = 90
    ys[ys>45] = 90
    ys[ys<=45] = 0 #TODO: consult: what should the threshold be
    ys[ys==90] = 1
    ys = closing(ys, rectangle(1,15))
    ys = pd.DataFrame(ys, index=index)
    ys.index.name = 'bee_id'
    return ys